# Experiment 2. Preliminary fingers-press asynchrony analysis

In [1]:
import os
import pandas as pd
import numpy as np
import os

In [2]:
### constants
data_folder= 'Experiment 2'
buttons= ['J', 'K']
event_code= ['Pressed', 'Released']
observerN= 15

In [6]:
### importing all responses into a single table
dfDict= {'ID': [], 'Time': [], 'Key': [], 'EventLabel': [], 'Target': [], 'Event': []}
for iO in range(1, observerN+1):
    with open(os.path.join(data_folder, 's%02d_FOA.txt'%(iO)), 'rt') as response_file:
        for current_line in response_file:
            # skipping the "start of the block" line
            if current_line[:len("--Activated")]=="--Activated":
                continue
                
            subfields= current_line.split()
            if subfields[-1][0] in buttons: # relevant button
                # adding the next line to the dictionary
                dfDict['ID'].append('s%02d'%(iO))

                # parsing time into seconds
                time_components= subfields[0].split(':') # assuming the hours:minutes:float_seconds
                if len(time_components)==1: # 
                    time_components= subfields[0].split('-') # nope, this is hours-minutes-float_seconds
                four_time_components= np.concatenate([time_components[0:-1], time_components[-1].split('.')])
                time_in_seconds= np.sum(np.array(four_time_components).astype('float')*np.array([60*60, 60, 1, 1/100.0]))
                dfDict['Time'].append(time_in_seconds)

                # parsing event into target and event type
                dfDict['Key'].append(subfields[-1][0])
                dfDict['EventLabel'].append(subfields[-1].strip()[2:-1])
                dfDict['Target'].append(buttons.index(subfields[-1][0]))
                dfDict['Event'].append(event_code.index(subfields[-1].strip()[2:-1]))
            
responses= pd.DataFrame(dfDict, columns=['ID', 'Time', 'Target', 'Event', 'Key', 'EventLabel'])
responses.to_csv(os.path.join('Experiment 2', 'Experiment2_FOA_Response.csv'), sep=';', index= False)

In [7]:
## computing distribution of response times for individual observers

dfDict= {'ID': [], 'Event':[], 'MaxRT':[], 'MeanRT':[], 'MinRT':[], 'StdRT':[]}

### computing response variance for each observer
for current_observer in responses.ID.unique():
    iTrials= responses[responses.ID==current_observer].index
    
    for current_event in range(2):
        # finding relevant double events
        iDoubleEvent= responses[(responses.ID==current_observer) & 
                                        (responses.Target[iTrials[:-1]]!=responses.Target[iTrials[1:]]) & 
                                        (responses.Event[iTrials[:-1]]==responses.Event[iTrials[1:]]) &
                                        (responses.Event[iTrials[:-1]]==current_event)].index
        rtDist= responses.Time[iDoubleEvent+1].values-responses.Time[iDoubleEvent].values
        
        # characterizing distribution
        dfDict['ID'].append(current_observer)
        dfDict['Event'].append(current_event)
        dfDict['MaxRT'].append(np.max(rtDist))
        dfDict['MinRT'].append(np.min(rtDist))
        dfDict['MeanRT'].append(np.mean(rtDist))
        dfDict['StdRT'].append(np.std(rtDist))
                                    
ResponseBrackets= pd.DataFrame(dfDict)
ResponseBrackets.to_csv(os.path.join('Experiment 2', 'Experiment2_FOA_Estimate.csv'), sep=';', index= False)